# 线性回归的简洁实现


通过使用深度学习框架来简洁实现 线性回归模型 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

c:\Users\Miao\.conda\envs\pytorch\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] 找不到指定的程序。
  warn(f"Failed to load image Python extension: {e}")


调用框架中现有的API来读取数据     
我们将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
# data_iter = load_array((features, labels), batch_size)

# next(iter(data_iter))

In [12]:

import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用框架定义好的层。   
这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。我们首先定义一个模型变量net，它是一个`Sequential`类的实例。`Sequential`类将多个层串联在一起。当给定输入数据时，`Sequential`实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要`Sequential`。但是由于以后几乎所有的模型都是多层的，这里使用`Sequential`会让你熟悉“标准的流水线”

In [13]:
# `nn`是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [15]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [20]:
loss = nn.MSELoss()

实例化`SGD`实例    
SGD(随机梯度下降):样本随机采样。        
当我们实例化一个`SGD`实例时，我们需要指定优化的参数(可通过`net.parameters()`从我们的模型中获得)以及优化算法所需要的超参数字典。小批量随机梯度下降只需要设置lr值。

In [17]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与之前的从零开始实现所做的非常相似。        
在每个训练周期里，我们将完整遍历一次数据集(train_data)，不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，我们会进行以下步骤：
- 通过调用`net(X)`生成预测并计算损失`l`(前向传播)
- 通过进行反向传播来计算梯度
- 通过调用优化器来更新模型参数

In [21]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch+1}, loss{l:f}')

epoch 1, loss0.000201
epoch 2, loss0.000093
epoch 3, loss0.000093
